In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
from datetime import datetime


DATA_DIR = os.path.expanduser("~/Desktop/AICombatCoach/data")
CLASSES = ['jab', 'cross', 'guard']
SAMPLES_PER_CLASS = 100  # You can change this

# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


for label in CLASSES:
    class_path = os.path.join(DATA_DIR, label)
    os.makedirs(class_path, exist_ok=True)

#collection
def collect_pose_data(label):
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        collected = 0
        last_saved_time = time.time()

        print(f"📸 Collecting for class: {label}")
        print("⚠️ Start posing. Press 'Esc' to exit early.")
        time.sleep(3)

        while cap.isOpened() and collected < SAMPLES_PER_CLASS:
            ret, frame = cap.read()
            if not ret:
                continue

            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

          
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

               
                keypoints = []
                for lm in results.pose_landmarks.landmark:
                    keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])

              
                if time.time() - last_saved_time >= 1:
                    npy_path = os.path.join(DATA_DIR, label, f"{label}_{collected}.npy")
                    np.save(npy_path, keypoints)
                    collected += 1
                    last_saved_time = time.time()

                    print(f"[{collected}/{SAMPLES_PER_CLASS}] Saved: {npy_path}")

            
            cv2.imshow('Collecting Pose Data', image)

            if cv2.waitKey(10) & 0xFF == 27:  # ESC key to quit
                break

        cap.release()
        cv2.destroyAllWindows()
        print(f"✅ Done collecting for class: {label}")


for label in CLASSES:
    collect_pose_data(label)


📸 Collecting for class: jab
⚠️ Start posing. Press 'Esc' to exit early.
[1/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_0.npy
[2/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_1.npy
[3/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_2.npy
[4/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_3.npy
[5/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_4.npy
[6/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_5.npy
[7/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_6.npy
[8/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_7.npy
[9/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_8.npy
[10/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_9.npy
[11/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_10.npy
[12/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_11.npy
[13/100] Saved: C:\Users\DELL/Desktop/AICombatCoach/data\jab\jab_12.npy
[14/100] Sa

In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


DATA_DIR = r'C:\Users\DELL\Desktop\AICombatCoach\data'
LABELS = ['jab', 'cross', 'guard']
label_map = {label: idx for idx, label in enumerate(LABELS)}


def load_pose_data():
    X, y = [], []
    for label in LABELS:
        folder = os.path.join(DATA_DIR, label)
        for file in os.listdir(folder):
            if file.endswith('.npy'):
                filepath = os.path.join(folder, file)
                try:
                    pose_array = np.load(filepath)
                    if pose_array.shape != (132,):
                        continue  # skip if data is not as expected
                    X.append(pose_array)
                    y.append(label_map[label])
                except Exception as e:
                    print(f"Failed to load {filepath}: {e}")
    return np.array(X), np.array(y)


X, y = load_pose_data()
print(f"X shape: {X.shape}, y shape: {y.shape}")
np.save("X.npy", X)
np.save("y.npy", y)


X shape: (300, 132), y shape: (300,)


In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib


X = np.load("X.npy")
y = np.load("y.npy")

print("X shape:", X.shape)  
print("y shape:", y.shape)  


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)


clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_flat, y_train)


y_pred = clf.predict(X_test_flat)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc * 100:.2f}%")


joblib.dump(clf, "pose_classifier.pkl")
print("Model saved as 'pose_classifier.pkl'")

X shape: (300, 132)
y shape: (300,)
Test Accuracy: 100.00%
Model saved as 'pose_classifier.pkl'


In [18]:
!pip install mediapipe opencv-python joblib


In [23]:
import cv2
import numpy as np
import mediapipe as mp
import joblib


model = joblib.load(r"C:\Users\DELL\Desktop\AICombatCoach\notebooks\pose_classifier.pkl")


label_map = {0: "cross", 1: "jab", 2: "guard"}


mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    
    image = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        
        landmarks = results.pose_landmarks.landmark
        pose_row = []
        for landmark in landmarks:
            pose_row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        
       
        pose_array = np.array([pose_row])

      
        prediction = model.predict(pose_array)[0]
        predicted_label = label_map.get(prediction, "Unknown")

        
        cv2.putText(image, f'Pose: {predicted_label}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Pose Classification', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
